# Environment and Simulation Setup
## Creating and Sourcing URDF Files

This tasks includes the meshes (`.stl` files), collision geometry, and `.urdf` files for each object. We plan to use the Franka panda robotic arm, which is the same arm purchased in our research group (the Computational Robotics Lab at Harvard).